# 1. Повнозв'язані нейронні мережі
Вирішіть завдання класифікації даних, з якими ви працювали в лабораторній № 1 за допомогою повнозв’язаної нейромережі прямого поширення (fully connected feed-forward network). Результати порівняйте з одержаними раніше. 

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

data = pd.read_csv('heart.csv')
data.head()

/Users/philip/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [2]:
# One-hot encoding для категоріальних змінних
categorical_columns = ['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope']
data_encoded = pd.get_dummies(data, columns=categorical_columns, drop_first=True)

# Нормалізація числових ознак
numerical_columns = ['Age', 'RestingBP', 'Cholesterol', 'MaxHR', 'Oldpeak']
scaler = StandardScaler()
data_encoded[numerical_columns] = scaler.fit_transform(data_encoded[numerical_columns])

# Відокремлення ознак та цільової змінної
X = data_encoded.drop(columns=['HeartDisease'])
y = data_encoded['HeartDisease']

# Розділення на навчальну і тестову вибірки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Перегляд розмірів вибірок
X_train.shape, X_test.shape, y_train.shape, y_test.shape


((642, 15), (276, 15), (642,), (276,))

Навчальна вибірка містить 642 зразки, тестова — 276. Всього використовується 15 ознак для передбачення наявності серцевих захворювань.

In [3]:
# Побудова повнозв'язаної нейронної мережі
model = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(16, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')  # Для бінарної класифікації
])

# Компіляція моделі
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Навчання моделі
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=16, verbose=1)

# Оцінка моделі на тестовій вибірці
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
test_loss, test_accuracy


Epoch 1/50


/Users/philip/Library/Python/3.9/lib/python/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5110 - loss: 0.7389 - val_accuracy: 0.7319 - val_loss: 0.5715
Epoch 2/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6797 - loss: 0.5923 - val_accuracy: 0.8080 - val_loss: 0.5075
Epoch 3/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7438 - loss: 0.5439 - val_accuracy: 0.8514 - val_loss: 0.4579
Epoch 4/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7798 - loss: 0.4978 - val_accuracy: 0.8514 - val_loss: 0.4117
Epoch 5/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8055 - loss: 0.4662 - val_accuracy: 0.8551 - val_loss: 0.3795
Epoch 6/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8196 - loss: 0.4539 - val_accuracy: 0.8587 - val_loss: 0.3592
Epoch 7/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8289 - loss: 0.4263 - val_accuracy: 0.8587 - val_loss: 0.3477
Epoch 8/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7996 - loss: 0.4414 - val_accuracy: 0.8623 - val_loss: 0.3380
Epo

(0.2884049713611603, 0.8876811861991882)

У порівнянні з, наприклад, Random Forest з лабораторної №1, точність моделі нейронної мережі така ж сама (88.76%). Пповнозв’язана нейронна мережа так само справляється з цим завданням класифікації.

# 2. Згорткові нейронні мережі
Вирішіть завдання класифікації зображень за допомогою згорткової (convolutional) нейромережі двома способами
а) навчить мережу з нуля (from scratch)
б) застосуйте перенесення навчання (transfer learning from pre-trained weights)

Порівняйте результати (якщо в обраному датасеті класів забагато, достатньо залишити 3-5).

In [18]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
import os

# Cats vs Dogs
data_dir = tf.keras.utils.get_file(
    'cats_and_dogs',
    'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip',
    extract=True)  # Завантаження та автоматичне розпакування архіву
data_dir = os.path.join(os.path.dirname(data_dir), 'cats_and_dogs/cats_and_dogs_filtered')
train_dir = f"{data_dir}/train"
validation_dir = f"{data_dir}/validation"

# Параметри для обробки зображень
batch_size = 32  # Кількість зображень у кожному пакеті
img_height = 150  # Висота зображення
img_width = 150  # Ширина зображення

# Генератори даних із нормалізацією значень пікселів
train_datagen = ImageDataGenerator(rescale=1.0/255.0)  # Нормалізація значень до діапазону [0, 1]
validation_datagen = ImageDataGenerator(rescale=1.0/255.0)

# Генерація даних для тренувального набору
train_ds = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),  # Зміна розміру зображень
    batch_size=batch_size,
    class_mode='binary'  # Цільова змінна є бінарною (cat/dog)
)

# Генерація даних для тестового набору
val_ds = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary'
)

# Навчання моделі з нуля
model_scratch = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),  # Перший згортковий шар
    MaxPooling2D(),  # Зменшення розмірності
    Conv2D(64, (3, 3), activation='relu'),  # Другий згортковий шар
    MaxPooling2D(),
    Conv2D(128, (3, 3), activation='relu'),  # Третій згортковий шар
    MaxPooling2D(),
    Flatten(),  # Розгортання в один вектор
    Dense(128, activation='relu'),  # Повнозв’язаний шар
    Dropout(0.5),  # Регуляризація
    Dense(1, activation='sigmoid')  # Вихідний шар для бінарної класифікації
])

# Компіляція моделі з оптимізатором Adam
model_scratch.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Навчання моделі
history_scratch = model_scratch.fit(train_ds, validation_data=val_ds, epochs=10)

# Перенесення навчання
base_model = MobileNetV2(input_shape=(img_height, img_width, 3), include_top=False, weights='imagenet')  # Використання попередньо навченої моделі
base_model.trainable = False  # Замороження ваг базової моделі

# Додавання нових шарів
x = Flatten()(base_model.output)  # Розгортання
x = Dense(128, activation='relu')(x)  # Новий повнозв’язаний шар
x = Dropout(0.5)(x)  # Регуляризація
output = Dense(1, activation='sigmoid')(x)  # Вихідний шар для бінарної класифікації
model_transfer = Model(inputs=base_model.input, outputs=output)

# Компіляція моделі з перенесеним навчанням
model_transfer.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Навчання моделі
history_transfer = model_transfer.fit(train_ds, validation_data=val_ds, epochs=10)

# Оцінка обох моделей
loss_scratch, acc_scratch = model_scratch.evaluate(val_ds)  # Оцінка моделі, навченої з нуля
loss_transfer, acc_transfer = model_transfer.evaluate(val_ds)  # Оцінка моделі з перенесеним навчанням

print(f"Навчання з нуля, точність = {acc_scratch}")
print(f"Перенесене навчання, точність = {acc_transfer}")


Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Epoch 1/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 17s 264ms/step - accuracy: 0.5245 - loss: 0.7605 - val_accuracy: 0.5530 - val_loss: 0.6848
Epoch 2/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 21s 341ms/step - accuracy: 0.5921 - loss: 0.6652 - val_accuracy: 0.6720 - val_loss: 0.6255
Epoch 3/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 23s 366ms/step - accuracy: 0.6704 - loss: 0.6190 - val_accuracy: 0.7120 - val_loss: 0.5773
Epoch 4/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 27s 431ms/step - accuracy: 0.7119 - loss: 0.5521 - val_accuracy: 0.7030 - val_loss: 0.5890
Epoch 5/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 27s 427ms/step - accuracy: 0.7476 - loss: 0.5102 - val_accuracy: 0.7000 - val_loss: 0.5846
Epoch 6/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 27s 430ms/step - accuracy: 0.7793 - loss: 0.4616 - val_accuracy: 0.7440 - val_loss: 0.5680
Epoch 7/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 29s 457ms/step - accuracy: 0.8412 - loss: 0.3573 - val_accuracy: 0.7350 - val_loss: 0.5789
Epoch 8/10
63

/var/folders/cq/lfznn8xn74x1ygrn_30hqgkm0000gn/T/ipykernel_89826/2913628136.py:61: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(input_shape=(img_height, img_width, 3), include_top=False, weights='imagenet')


Epoch 1/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 16s 223ms/step - accuracy: 0.8781 - loss: 1.6041 - val_accuracy: 0.9580 - val_loss: 0.5416
Epoch 2/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 15s 238ms/step - accuracy: 0.9609 - loss: 0.3215 - val_accuracy: 0.9430 - val_loss: 0.1689
Epoch 3/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 19s 303ms/step - accuracy: 0.9736 - loss: 0.0712 - val_accuracy: 0.9520 - val_loss: 0.1520
Epoch 4/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 15s 242ms/step - accuracy: 0.9766 - loss: 0.0888 - val_accuracy: 0.9440 - val_loss: 0.1728
Epoch 5/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 18s 285ms/step - accuracy: 0.9790 - loss: 0.0639 - val_accuracy: 0.9500 - val_loss: 0.1657
Epoch 6/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 18s 287ms/step - accuracy: 0.9856 - loss: 0.0469 - val_accuracy: 0.9570 - val_loss: 0.1860
Epoch 7/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 17s 269ms/step - accuracy: 0.9859 - loss: 0.0395 - val_accuracy: 0.9530 - val_loss: 0.1877
Epoch 8/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 18s 288ms/step - accuracy: 0.9875 - loss: 0.0286 - val_accu

У випадку навчання з нуля точність моделі поступово зростала протягом 10 епох, досягнувши максимуму в 69.1% на тестовій вибірці. Незважаючи на стабільне зменшення втрат на тренувальній вибірці, модель показала менш стабільну продуктивність на валідаційних даних.

Але модель із використанням перенесеного навчання (на основі MobileNetV2) показує значно вищу точність — 95.1% на тестовій вибірці. Завдяки вже навченим базовим вагам, модель швидко досягла високої результативності з низькими втратами.

In [26]:
import re
import string
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional
import tensorflow_hub as hub

# Завантаження даних
data = pd.read_csv('ecommerceDataset.csv')

# Перейменування колонок для зручності
data.columns = ['Category', 'Description']

# Видалення HTML-розмітки, пунктуації та очищення тексту
def preprocess_text(text):
    text = re.sub(r'<.*?>', '', text)  # Видалення HTML
    text = text.lower()  # Приведення до нижнього регістру
    text = text.translate(str.maketrans('', '', string.punctuation))  # Видалення пунктуації
    return text

data['Cleaned_Description'] = data['Description'].astype(str).apply(preprocess_text)

# Кодування міток класів
encoder = LabelEncoder()
data['Encoded_Category'] = encoder.fit_transform(data['Category'])

# Розділення даних
X = data['Cleaned_Description']
y = data['Encoded_Category']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Токенізація текстів
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Паддінг для вирівнювання довжини
max_len = 100
X_train_padded = pad_sequences(X_train_seq, maxlen=max_len, padding='post')
X_test_padded = pad_sequences(X_test_seq, maxlen=max_len, padding='post')

# Навчання з нуля 
vocab_size = 10000
embedding_dim = 100

model_scratch = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_len),
    Bidirectional(LSTM(64, return_sequences=False)),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dense(len(encoder.classes_), activation='softmax')
])

model_scratch.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history_scratch = model_scratch.fit(X_train_padded, y_train, validation_data=(X_test_padded, y_test), epochs=10, batch_size=32)

# Перенесене навчання з embeddings
embed = hub.load('https://tfhub.dev/google/nnlm-en-dim50/2')

# Перетворення текстів у вектори 
def embed_texts(texts):
    return np.array([embed([text]).numpy().flatten() for text in texts])

X_train_embedded = embed_texts(X_train)
X_test_embedded = embed_texts(X_test)

model_hub = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_embedded.shape[1],)),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(len(encoder.classes_), activation='softmax')
])

model_hub.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history_hub = model_hub.fit(X_train_embedded, y_train, validation_data=(X_test_embedded, y_test), epochs=10, batch_size=32)

# Оцінка моделей
loss_scratch, acc_scratch = model_scratch.evaluate(X_test_padded, y_test)
loss_hub, acc_hub = model_hub.evaluate(X_test_embedded, y_test)

print(f"Навчання з нуля: точність = {acc_scratch}")
print(f"Перенесене навчання з embeddings: точність = {acc_hub}")


Epoch 1/10


/Users/philip/Library/Python/3.9/lib/python/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


1103/1103 ━━━━━━━━━━━━━━━━━━━━ 60s 53ms/step - accuracy: 0.7970 - loss: 0.5324 - val_accuracy: 0.9597 - val_loss: 0.1528
Epoch 2/10
1103/1103 ━━━━━━━━━━━━━━━━━━━━ 61s 55ms/step - accuracy: 0.9756 - loss: 0.0995 - val_accuracy: 0.9609 - val_loss: 0.1442
Epoch 3/10
1103/1103 ━━━━━━━━━━━━━━━━━━━━ 65s 59ms/step - accuracy: 0.9842 - loss: 0.0616 - val_accuracy: 0.9591 - val_loss: 0.1585
Epoch 4/10
1103/1103 ━━━━━━━━━━━━━━━━━━━━ 67s 61ms/step - accuracy: 0.9904 - loss: 0.0367 - val_accuracy: 0.9621 - val_loss: 0.1639
Epoch 5/10
1103/1103 ━━━━━━━━━━━━━━━━━━━━ 59s 54ms/step - accuracy: 0.9910 - loss: 0.0306 - val_accuracy: 0.9668 - val_loss: 0.1494
Epoch 6/10
1103/1103 ━━━━━━━━━━━━━━━━━━━━ 59s 53ms/step - accuracy: 0.9933 - loss: 0.0243 - val_accuracy: 0.9690 - val_loss: 0.1778
Epoch 7/10
1103/1103 ━━━━━━━━━━━━━━━━━━━━ 59s 54ms/step - accuracy: 0.9952 - loss: 0.0176 - val_accuracy: 0.9669 - val_loss: 0.1876
Epoch 8/10
1103/1103 ━━━━━━━━━━━━━━━━━━━━ 58s 53ms/step - accuracy: 0.9945 - loss: 0.01

/Users/philip/Library/Python/3.9/lib/python/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1103/1103 ━━━━━━━━━━━━━━━━━━━━ 1s 822us/step - accuracy: 0.7603 - loss: 0.6606 - val_accuracy: 0.8995 - val_loss: 0.3264
Epoch 2/10
1103/1103 ━━━━━━━━━━━━━━━━━━━━ 1s 752us/step - accuracy: 0.8922 - loss: 0.3606 - val_accuracy: 0.9040 - val_loss: 0.3051
Epoch 3/10
1103/1103 ━━━━━━━━━━━━━━━━━━━━ 1s 748us/step - accuracy: 0.8991 - loss: 0.3397 - val_accuracy: 0.9086 - val_loss: 0.2959
Epoch 4/10
1103/1103 ━━━━━━━━━━━━━━━━━━━━ 1s 752us/step - accuracy: 0.9031 - loss: 0.3279 - val_accuracy: 0.9082 - val_loss: 0.2933
Epoch 5/10
1103/1103 ━━━━━━━━━━━━━━━━━━━━ 1s 787us/step - accuracy: 0.9055 - loss: 0.3298 - val_accuracy: 0.9134 - val_loss: 0.2840
Epoch 6/10
1103/1103 ━━━━━━━━━━━━━━━━━━━━ 1s 751us/step - accuracy: 0.9068 - loss: 0.3115 - val_accuracy: 0.9151 - val_loss: 0.2758
Epoch 7/10
1103/1103 ━━━━━━━━━━━━━━━━━━━━ 1s 763us/step - accuracy: 0.9094 - loss: 0.3052 - val_accuracy: 0.9170 - val_loss: 0.2715
Epoch 8/10
1103/1103 ━━━━━━━━━━━━━━━━━━━━ 1s 756us/step - accuracy: 0.9135 - loss: 0.29

 У першому підході модель використовувала двонаправлений шар LSTM з вбудованим шаром ембедингів, навчаючись з нуля. За допомогою цього підходу ми досягли високу точність на тестових даних – 97%. Модель має такі результаті великій кількості параметрів і здатності адаптуватися до специфіки текстів у даному датасеті.

У другому підході використовувалися готові векторні представлення слів із TensorFlow Hub (nnlm-en-dim50). Цей метод дозволив зменшити час навчання та забезпечив хорошу точність – 92%. Попередньо навчені ембедінги ефективно перенесли знання з великих текстових корпусів, що позитивно вплинуло на результати, хоча модель показала трохи нижчу точність, ніж підхід із навчанням з нуля. Таким чином, вибір підходу залежить від наявності ресурсів і часу: навчання з нуля підходить для завдань із достатньою кількістю даних, а перенесене навчання дозволяє швидко отримати прийнятні результати.